### file size 확인

In [1]:
import open3d as o3d

In [13]:
def count_points(ply_file_path):
    pcd = o3d.io.read_point_cloud(ply_file_path)
    filename = ply_file_path.split("/")[-1].split(".")[0]
    print(f"{filename} 포인트 수: {len(pcd.points)}")

In [15]:
import os 

def check_ply_file_size(file_path):
    # 파일이 존재하는지 확인
    if not os.path.exists(file_path):
        return "파일이 존재하지 않습니다."
    
    # 파일 크기 확인 (바이트 단위)
    file_size_bytes = os.path.getsize(file_path)
    
    filename = file_path.split("/")[-1]
    
    # 바이트를 적절한 단위로 변환
    if file_size_bytes < 1024:  # 1KB 미만
        result = f"{filename} {file_size_bytes} Byte"
    elif file_size_bytes < 1024 * 1024:  # 1MB 미만
        result = f"{filename} {file_size_bytes / 1024:.2f} KB"
    elif file_size_bytes < 1024 * 1024 * 1024:  # 1GB 미만
        result = f"{filename} {file_size_bytes / (1024 * 1024):.2f} MB"
    else:  # 1GB 이상
        result = f"{filename} {file_size_bytes / (1024 * 1024 * 1024):.2f} GB"
    
    count_points(file_path)
    print(result)
    # return result

In [10]:
count_points("./data/gate.ply")
count_points("./data/shoe.ply")

gate 포인트 수: 14849508
shoe 포인트 수: 7983083


### read & visualization point cloud

In [8]:
def visualize_ply(ply_file_path):
    # PCD 파일 읽기
    pcd = o3d.io.read_point_cloud(ply_file_path)
    # 포인트 클라우드 정보 출력
    print(f"포인트 수: {len(pcd.points)}")
    print(f"포인트 클라우드 특성:")
    print(pcd)
    
    # 포인트 클라우드에 색상이 없는 경우 임의의 색상 지정
    if not pcd.has_colors():
        pcd.paint_uniform_color([0.5, 0.5, 0.5])  # 회색으로 지정
    
    # 시각화
    o3d.visualization.draw([pcd])

# 사용 예시
ply_file_path = "./data/gate_voxelized_0.05.ply"
visualize_ply(ply_file_path)

포인트 수: 776198
포인트 클라우드 특성:
PointCloud with 776198 points.


### voxelize

In [10]:
import open3d as o3d
import numpy as np

def voxelize_ply(input_ply_path, output_ply_path, voxel_size=0.05, 
                 min_bound=None, max_bound=None, 
                 linear_interpolate=False, 
                 crop=True):
    """
    PLY 파일을 읽어 voxel화하고 결과를 PLY 파일로 저장합니다.

    매개변수:
    - input_ply_path: 입력 PLY 파일 경로
    - output_ply_path: 출력 PLY 파일 경로
    - voxel_size: voxel의 크기 (기본값: 0.05)
    - min_bound: voxel 그리드의 최소 경계 (기본값: None, 자동 계산)
    - max_bound: voxel 그리드의 최대 경계 (기본값: None, 자동 계산)
    - linear_interpolate: 선형 보간 사용 여부 (기본값: False)
    - crop: 원본 포인트 클라우드 범위로 voxel 그리드를 자르기 (기본값: True)
    """
    
    # PLY 파일 읽기
    print(f"PLY 파일 읽는 중: {input_ply_path}")
    pcd = o3d.io.read_point_cloud(input_ply_path)
    
    # 원본 포인트 클라우드 정보 출력
    print(f"원본 포인트 수: {len(pcd.points)}")
    
    # Voxel 그리드 생성
    print(f"Voxel 크기: {voxel_size}")
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)
    
    # Voxel 그리드를 포인트 클라우드로 변환
    voxel_pcd = o3d.geometry.PointCloud()
    voxel_points = []
    voxel_colors = []
    
    # Voxel 순회
    for voxel in voxel_grid.get_voxels():
        voxel_points.append(voxel.grid_index.astype(float) * voxel_size + voxel_grid.origin)
        voxel_colors.append(voxel.color)
    
    voxel_pcd.points = o3d.utility.Vector3dVector(np.array(voxel_points))
    voxel_pcd.colors = o3d.utility.Vector3dVector(np.array(voxel_colors))
    
    # Voxel화된 포인트 클라우드 정보 출력
    print(f"Voxel화된 포인트 수: {len(voxel_pcd.points)}")
    
    # 결과 저장
    print(f"결과 저장 중: {output_ply_path}")
    o3d.io.write_point_cloud(output_ply_path, voxel_pcd)
    
    print("Voxel화 완료")
    
    return voxel_pcd

# 사용 예시
voxel_size = 1.0
object_name = "gate"
input_ply_path = f"./data/{object_name}.ply"
output_ply_path = f"./data/{object_name}_voxelized_{voxel_size}.ply"

# Voxel화 실행
voxelized_pcd = voxelize_ply(input_ply_path, output_ply_path, voxel_size=voxel_size)

# 결과 시각화 (선택사항)
o3d.visualization.draw([voxelized_pcd])

PLY 파일 읽는 중: ./data/gate.ply
원본 포인트 수: 14849508
Voxel 크기: 1.0
Voxel화된 포인트 수: 1703
결과 저장 중: ./data/gate_voxelized_1.0.ply
Voxel화 완료


### voxel block 시각화

In [11]:
import open3d as o3d
import numpy as np

def visualize_voxelized_ply_as_blocks(input_ply_path, block_size=1.0):
    """
    이미 복셀화된 PLY 파일을 읽어 블록 형태로 시각화합니다.

    매개변수:
    - input_ply_path: 입력 복셀화된 PLY 파일 경로
    - block_size: 각 블록의 크기 (기본값: 1.0)
    """
    # PLY 파일 읽기
    pcd = o3d.io.read_point_cloud(input_ply_path)
    print(f"포인트(복셀) 수: {len(pcd.points)}")

    # 포인트를 블록으로 변환
    blocks = []
    
    for point, color in zip(pcd.points, pcd.colors):
        # 큐브 생성
        cube = o3d.geometry.TriangleMesh.create_box(width=block_size, 
                                                    height=block_size, 
                                                    depth=block_size)
        cube.translate(point - np.array([block_size/2, block_size/2, block_size/2]))
        
        # 큐브 색상 설정
        cube.paint_uniform_color(color)
        
        blocks.append(cube)
    
    # 모든 블록 합치기
    combined_blocks = blocks[0]
    for block in blocks[1:]:
        combined_blocks += block
    
    print(f"생성된 블록 수: {len(blocks)}")
    
    # 시각화
    o3d.visualization.draw([combined_blocks])

    return combined_blocks

# 사용 예시
object_name = "gate"
voxel_size = 1.0
input_ply_path = f"./data/{object_name}_voxelized_{voxel_size}.ply"
block_size = voxel_size  # 블록 크기 조정

voxelized_blocks = visualize_voxelized_ply_as_blocks(input_ply_path, block_size)

포인트(복셀) 수: 1703
생성된 블록 수: 1703


In [10]:
print("원본 size")
check_ply_file_size("./data/gate.ply")
check_ply_file_size("./data/shoe.ply")


원본 size
gate 포인트 수: 14849508
gate.ply 551.54 MB
shoe 포인트 수: 7983083
shoe.ply 205.56 MB


In [17]:
voxel_size = 1.0
print(f"voxel size {voxel_size}")
check_ply_file_size(f"./data/gate_voxelized_{voxel_size}.ply")
check_ply_file_size(f"./data/shoe_voxelized_{voxel_size}.ply")

voxel size 1.0
gate_voxelized_1 포인트 수: 1703
gate_voxelized_1.0.ply 45.11 KB


'파일이 존재하지 않습니다.'

In [19]:
voxel_size = 0.01
print(f"voxel size {voxel_size}")
check_ply_file_size(f"./data/gate_voxelized_{voxel_size}.ply")
check_ply_file_size(f"./data/shoe_voxelized_{voxel_size}.ply")

voxel size 0.01
gate_voxelized_0 포인트 수: 10645535
gate_voxelized_0.01.ply 274.11 MB
shoe_voxelized_0 포인트 수: 46930
shoe_voxelized_0.01.ply 1.21 MB


In [20]:
voxel_size = 0.05
print(f"voxel size {voxel_size}")
check_ply_file_size(f"./data/gate_voxelized_{voxel_size}.ply")
check_ply_file_size(f"./data/shoe_voxelized_{voxel_size}.ply")

voxel size 0.05
gate_voxelized_0 포인트 수: 776198
gate_voxelized_0.05.ply 19.99 MB
shoe_voxelized_0 포인트 수: 1602
shoe_voxelized_0.05.ply 42.44 KB


In [21]:
voxel_size = 0.1
print(f"voxel size {voxel_size}")
check_ply_file_size(f"./data/gate_voxelized_{voxel_size}.ply")
check_ply_file_size(f"./data/shoe_voxelized_{voxel_size}.ply")

voxel size 0.1
gate_voxelized_0 포인트 수: 194795
gate_voxelized_0.1.ply 5.02 MB
shoe_voxelized_0 포인트 수: 364
shoe_voxelized_0.1.ply 9.80 KB


### 포인트 클라우드 정합

In [16]:
import open3d as o3d
import numpy as np

def scale_point_cloud(pcd, scale_factor):
    """
    포인트 클라우드의 크기를 조정합니다.
    """
    points = np.asarray(pcd.points)
    center = np.mean(points, axis=0)
    points = (points - center) * scale_factor + center
    scaled_pcd = o3d.geometry.PointCloud()
    scaled_pcd.points = o3d.utility.Vector3dVector(points)
    if pcd.has_colors():
        scaled_pcd.colors = pcd.colors
    if pcd.has_normals():
        scaled_pcd.normals = pcd.normals
    return scaled_pcd

def align_centers(source, target):
    """
    두 포인트 클라우드의 중심을 맞춥니다.
    """
    source_center = source.get_center()
    target_center = target.get_center()
    
    translation = target_center - source_center
    
    # source를 target의 중심으로 이동
    source.translate(translation)
    
    return source, target

def register_point_clouds(source_path, target_path, output_path, scale_factor=5.0):
    # 포인트 클라우드 로드
    source = o3d.io.read_point_cloud(source_path)
    target = o3d.io.read_point_cloud(target_path)

    # 신발(source) 크기 조정
    source = scale_point_cloud(source, scale_factor)

    # 중심 맞추기
    source, target = align_centers(source, target)

    # 초기 정합을 위한 전처리
    source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1 * scale_factor, max_nn=30))
    target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    # 전역 정합 (Global Registration)
    # FPFH 특징 계산
    source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(source, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25 * scale_factor, max_nn=100))
    target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(target, o3d.geometry.KDTreeSearchParamHybrid(radius=0.25, max_nn=100))

    # RANSAC을 이용한 전역 정합
    result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, source_fpfh, target_fpfh, True,
        0.05 * scale_factor,  # 최대 대응 거리
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, # RANSAC 모델 파라미터 수
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(0.05 * scale_factor)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(1000000, 0.999))

    # 정합 결과 적용
    source.transform(result_ransac.transformation)

    # ICP를 이용한 미세 정합
    result_icp = o3d.pipelines.registration.registration_icp(
        source, target, 0.05 * scale_factor, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

    # 최종 정합 결과 적용
    source.transform(result_icp.transformation)

    # 정합된 source를 원래 크기로 되돌리기
    source = scale_point_cloud(source, 1/scale_factor)

    # 정합된 포인트 클라우드 합치기
    combined = source + target

    # 결과 저장
    o3d.io.write_point_cloud(output_path, combined)

    return combined

def visualize_point_cloud(pcd):
    o3d.visualization.draw([pcd])

# 사용 예시
source_path = f"./data/shoe_voxelized_0.01.ply"
target_path = f"./data/gate_voxelized_0.05.ply"
scale_factor = 10.0
output_path = f"./data/combined_output_{scale_factor}.ply"

# 정합 수행 및 결과 저장
combined_pcd = register_point_clouds(source_path, target_path, output_path, scale_factor=scale_factor)

# 결과 시각화
visualize_point_cloud(combined_pcd)

### 스케일링

In [ ]:
def scale_point_cloud(pcd, scale_factor):
    """
    Point cloud를 주어진 스케일 팩터만큼 확대 또는 축소합니다.
    
    :param pcd: Open3D PointCloud 객체
    :param scale_factor: 스케일링 팩터 (1보다 크면 확대, 작으면 축소)
    :return: 스케일링된 Open3D PointCloud 객체
    """
    # 포인트 클라우드의 점들을 NumPy 배열로 가져옵니다
    points = np.asarray(pcd.points)
    
    # 각 점의 좌표에 스케일 팩터를 곱합니다
    scaled_points = points * scale_factor
    
    # 새로운 Open3D PointCloud 객체를 생성합니다
    scaled_pcd = o3d.geometry.PointCloud()
    
    # 스케일링된 점들을 새 PointCloud 객체에 설정합니다
    scaled_pcd.points = o3d.utility.Vector3dVector(scaled_points)
    
    return scaled_pcd

# 사용 예:
pcd = o3d.io.read_point_cloud("./data/shoe.ply")
scaled_pcd = scale_point_cloud(pcd, 1.5)  # 1.5배 확대